# Libraries

In [1]:
# Basics
import pandas as pd 
import numpy as np 
import io
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

# Others
import openpyxl
import xlrd
from pathlib import Path
import os
import warnings
from math import isnan
import category_encoders as ce

In [2]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning) 
warnings.filterwarnings("ignore", category = FutureWarning) 
warnings.filterwarnings("ignore", category = UserWarning) 

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Import Dataset 

In [4]:
path_in = r'C:\Users\luand\Documents\Projetos de ML\Projetos de ML Origo\Priorização de setores\RAIS 2021'

In [5]:
a = os.listdir(path_in)
df = pd.DataFrame(list(a), columns=['name_file'])
df[['file', 'extension']] = df['name_file'].str.rsplit('.', n=1, expand=True)
df = df[(df['extension'] == 'txt')]
df = df.reset_index()
df.head()

,index,name_file,file,extension
0,2,RAIS_VINC_PUB_MG_ES_RJ.txt,RAIS_VINC_PUB_MG_ES_RJ,txt


In [ ]:
name_file_df = df.iloc[0]['name_file']
file_in = os.path.join(path_in, name_file_df)
FileObjTxtv2 = pd.read_csv(file_in, sep=";", encoding='latin-1')

In [7]:
FileObjTxtv2.shape

(14435612, 60)

# Processing Dataset

In [8]:
FileObjTxtv2 = FileObjTxtv2[(FileObjTxtv2['Vínculo Ativo 31/12'] == 1)]
FileObjTxtv2.shape

(10029292, 60)

In [9]:
FileObjTxtv2.columns

Index(['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'Causa Afastamento 1',
       'Causa Afastamento 2', 'Causa Afastamento 3', 'Motivo Desligamento',
       'CBO Ocupação 2002', 'CNAE 2.0 Classe', 'CNAE 95 Classe',
       'Distritos SP', 'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 'Mês Admissão', 'Mês Desligamento',
       'Mun Trab', 'Município', 'Nacionalidade', 'Natureza Jurídica',
       'Ind Portador Defic', 'Qtd Dias Afastamento', 'Raça Cor',
       'Regiões Adm DF', 'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)',
       'Vl Remun Média Nom', 'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse',
       'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tempo Emprego',
       'Tipo Admissão', 'Tipo Estab', 'Tipo Estab.1', 'Tipo Defic',
       'Tipo Vínculo', 'IBGE Subsetor', 'Vl 

In [10]:
FileObjTxtv2 = FileObjTxtv2[[
       'CBO Ocupação 2002', 
       'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 
       'Município', 'Natureza Jurídica',
       'Ind Portador Defic', 'Qtd Dias Afastamento', 'Raça Cor',
       'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)',
       'Vl Remun Média Nom', 'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse',
       'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tempo Emprego',
       'Tipo Admissão', 'Tipo Defic',
       'Tipo Vínculo', 'IBGE Subsetor', 'Vl Rem Janeiro SC',
       'Vl Rem Fevereiro SC', 'Vl Rem Março SC', 'Vl Rem Abril SC',
       'Vl Rem Maio SC', 'Vl Rem Junho SC', 'Vl Rem Julho SC',
       'Vl Rem Agosto SC', 'Vl Rem Setembro SC', 'Vl Rem Outubro SC',
       'Vl Rem Novembro SC', 'Ind Trab Intermitente',
       'Ind Trab Parcial']]
FileObjTxtv2.shape

(10029292, 42)

In [11]:
# Selecionado apenas as Divisões da Indústria de Transformação
lst_IND = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
FileObjTxtv2['DIV_CNAE'] = round(((FileObjTxtv2['CNAE 2.0 Subclasse']/100000) - ((FileObjTxtv2['CNAE 2.0 Subclasse']%100000)/100000)), 4)
FileObjTxtv2 = FileObjTxtv2[(FileObjTxtv2['DIV_CNAE'].isin(lst_IND))]
FileObjTxtv2.shape

(1223051, 43)

In [12]:
Subclasse = FileObjTxtv2.groupby(['CNAE 2.0 Subclasse']).agg(N = ('Vínculo Ativo 31/12', 'count'))
Subclasse = Subclasse.sort_values(['N'], ascending = [False]).reset_index()
Subclasse.head(50)

,CNAE 2.0 Subclasse,N
0,1412601,51429
1,3101200,35166
2,1052000,30783
3,2422901,26157
4,1012101,25568
5,1091102,25238
6,1411801,24253
7,2949299,23271
8,2391503,21693
9,1071600,20905


In [13]:
Subclasse.shape

(405, 2)

In [14]:
# Subclasse = Subclasse[(Subclasse['N'] > 30000)]
# Subclasse = Subclasse[(Subclasse['N'] < 85000)]
# Subclasse['N'].sum(), Subclasse.shape

In [10]:
# name_list = Subclasse['CNAE 2.0 Subclasse'].values.tolist()

In [11]:
# FileObjTxtv2 = FileObjTxtv2[(FileObjTxtv2['CNAE 2.0 Subclasse'].isin(name_list))]
# FileObjTxtv2.shape

(2114982, 43)

In [15]:
FileObjTxtv2.columns = FileObjTxtv2.columns.str.replace(" ", "_", regex=True)
FileObjTxtv2.columns

Index(['CBO_Ocupação_2002', 'Vínculo_Ativo_31/12', 'Faixa_Etária',
       'Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)',
       'Faixa_Remun_Média_(SM)', 'Faixa_Tempo_Emprego',
       'Escolaridade_após_2005', 'Qtd_Hora_Contr', 'Idade',
       'Ind_CEI_Vinculado', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor',
       'Vl_Remun_Dezembro_Nom', 'Vl_Remun_Dezembro_(SM)', 'Vl_Remun_Média_Nom',
       'Vl_Remun_Média_(SM)', 'CNAE_2.0_Subclasse', 'Sexo_Trabalhador',
       'Tamanho_Estabelecimento', 'Tempo_Emprego', 'Tipo_Admissão',
       'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Vl_Rem_Janeiro_SC',
       'Vl_Rem_Fevereiro_SC', 'Vl_Rem_Março_SC', 'Vl_Rem_Abril_SC',
       'Vl_Rem_Maio_SC', 'Vl_Rem_Junho_SC', 'Vl_Rem_Julho_SC',
       'Vl_Rem_Agosto_SC', 'Vl_Rem_Setembro_SC', 'Vl_Rem_Outubro_SC',
       'Vl_Rem_Novembro_SC', 'Ind_Trab_Intermitente', 'Ind_Trab_Parcial',
       'DIV_CNAE'],
      dtype='object')

In [16]:
FileObjTxtv2 = FileObjTxtv2[['CBO_Ocupação_2002', 'Vínculo_Ativo_31/12', 'Faixa_Etária', 'Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)',
                             'Faixa_Remun_Média_(SM)', 'Faixa_Tempo_Emprego', 'Escolaridade_após_2005', 'Qtd_Hora_Contr', 'Idade', 'Ind_Simples', 
                             'Município', 'Natureza_Jurídica', 'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor',
                             'Vl_Remun_Dezembro_Nom', 'Vl_Remun_Dezembro_(SM)', 'Vl_Remun_Média_Nom', 'Vl_Remun_Média_(SM)', 'CNAE_2.0_Subclasse',
                             'Sexo_Trabalhador', 'Tamanho_Estabelecimento', 'Tempo_Emprego', 'Tipo_Admissão', 'Tipo_Defic',
                             'Tipo_Vínculo', 'IBGE_Subsetor', 
                             'Vl_Rem_Janeiro_SC', 'Vl_Rem_Fevereiro_SC', 'Vl_Rem_Março_SC', 'Vl_Rem_Abril_SC', 'Vl_Rem_Maio_SC', 'Vl_Rem_Junho_SC', 
                             'Vl_Rem_Julho_SC', 'Vl_Rem_Agosto_SC', 'Vl_Rem_Setembro_SC', 'Vl_Rem_Outubro_SC', 'Vl_Rem_Novembro_SC', 
                             'Ind_Trab_Intermitente', 'Ind_Trab_Parcial']]

In [17]:
file_path = Path(r"C:\Users\luand\Documents\Projetos de ML\Salary-Prediction\RAIS_VINC_PUB_MG_ES_RJ_Vinculo_0.csv")
FileObjTxtv2.to_csv(file_path, sep=';', index=False)